In [106]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net

In [107]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-18 18:49:45--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.29MB/s    in 0.2s    

2022-09-18 18:49:45 (5.29 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [108]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [113]:
#import csv from aws bucket
from pyspark import SparkFiles
url = "https://hhpp-bucket.s3.amazonaws.com/HoustonSuperNeighborhoods.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("HoustonSuperNeighborhoods.csv"), sep=",", header=True, inferSchema=True)
df.show()

+---+--------------------+----------------+-----------------------+
| id|                name|total_population|median_household_income|
+---+--------------------+----------------+-----------------------+
|  1|    Kashmere Gardens|         10055.0|                23102.0|
|  2|Trinity / Houston...|         15789.0|                27789.0|
|  3|           Settegast|          2981.0|                28821.0|
|  4|Denver Harbor / P...|         17571.0|                29273.0|
|  5|          South Park|         21208.0|                29475.0|
|  6|            Westwood|         19530.0|                29527.0|
|  7|  Greater Fifth Ward|         19687.0|                30535.0|
|  8|Clinton Park / Tr...|          3140.0|                31826.0|
|  9|        East Houston|         18580.0|                31826.0|
| 10|       Magnolia Park|         16999.0|                32039.0|
| 11|Gulfgate Rivervie...|         12723.0|                32595.0|
| 12|       Pleasantville|          2860.0|     

In [ ]:
#gathering total_population information to find mean and add to data
df_describe =df.select(["total_population"])
df_describe.show()


+----------------+
|total_population|
+----------------+
|         10055.0|
|         15789.0|
|          2981.0|
|         17571.0|
|         21208.0|
|         19530.0|
|         19687.0|
|          3140.0|
|         18580.0|
|         16999.0|
|         12723.0|
|          2860.0|
|         25724.0|
|         77220.0|
|         59410.0|
|         41392.0|
|         19141.0|
|          3569.0|
|         16259.0|
|         51432.0|
+----------------+
only showing top 20 rows



In [ ]:
#taking mean and creating new row in data. Adding mean for total_population of no super neighborhood row
df_describe.describe().show()

+-------+------------------+
|summary|  total_population|
+-------+------------------+
|  count|                88|
|   mean|24819.045454545456|
| stddev|20102.476331078928|
|    min|            1951.0|
|    max|          106657.0|
+-------+------------------+



In [ ]:
#after adding row 0 with new data we will upload the data from csv to database
url = "https://hhpp-bucket.s3.amazonaws.com/HoustonSuperNeighborhoodsUpdated.csv"
spark.sparkContext.addFile(url)
df_table = spark.read.csv(SparkFiles.get("HoustonSuperNeighborhoodsUpdated.csv"), sep=",", header=True, inferSchema=True)
df_table.show()

+---+--------------------+----------------+-----------------------+
| id|                name|total_population|median_household_income|
+---+--------------------+----------------+-----------------------+
|  0|                 N/A|           24819|                  30438|
|  1|    Kashmere Gardens|           10055|                  23102|
|  2|Trinity / Houston...|           15789|                  27789|
|  3|           Settegast|            2981|                  28821|
|  4|Denver Harbor / P...|           17571|                  29273|
|  5|          South Park|           21208|                  29475|
|  6|            Westwood|           19530|                  29527|
|  7|  Greater Fifth Ward|           19687|                  30535|
|  8|Clinton Park / Tr...|            3140|                  31826|
|  9|        East Houston|           18580|                  31826|
| 10|       Magnolia Park|           16999|                  32039|
| 11|Gulfgate Rivervie...|           12723|     

In [201]:
#Check datatype
df_table.printSchema

<bound method DataFrame.printSchema of DataFrame[id: int, name: string, total_population: int, median_household_income: int]>

In [187]:
# Store environmental variable

from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS

mode = "append"
jdbc_url="jdbc:postgresql://hhpp-db.cib8i0dtf6rx.us-east-1.rds.amazonaws.com:5432/hhppdb"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write df to super neighborhoods table in RDS
df_table.write.jdbc(url=jdbc_url, table='houston_super_neighborhoods', mode=mode, properties=config)

In [188]:
# Read super neighborhoods table from database
read_df = spark.read.jdbc(url=jdbc_url, table='houston_super_neighborhoods', properties=config)
read_df.show()

+---+--------------------+----------------+-----------------------+
| id|                name|total_population|median_household_income|
+---+--------------------+----------------+-----------------------+
|  0|                 N/A|           24819|                  30438|
|  1|    Kashmere Gardens|           10055|                  23102|
|  2|Trinity / Houston...|           15789|                  27789|
|  3|           Settegast|            2981|                  28821|
|  4|Denver Harbor / P...|           17571|                  29273|
|  5|          South Park|           21208|                  29475|
|  6|            Westwood|           19530|                  29527|
|  7|  Greater Fifth Ward|           19687|                  30535|
|  8|Clinton Park / Tr...|            3140|                  31826|
|  9|        East Houston|           18580|                  31826|
| 10|       Magnolia Park|           16999|                  32039|
| 11|Gulfgate Rivervie...|           12723|     

In [191]:
from IPython.utils import encoding
#import the properties csv
url = "https://hhpp-bucket.s3.amazonaws.com/Cleaned_Properties_IDs_fixed.csv"
spark.sparkContext.addFile(url)
df_properties = spark.read.csv(SparkFiles.get("Cleaned_Properties_IDs_fixed.csv"), sep=",", header=True, inferSchema=True)
df_properties.show()

+---+---+--------------------+--------+--------------------+---+-----+----+-------+----+
|_c0| id|             address|   price|                type|bed| sqft|walk|transit|bike|
+---+---+--------------------+--------+--------------------+---+-----+----+-------+----+
|  0| 80|4031 Wilderness F...|  369900|Single family res...|  2| 2395|  53|      0|  47|
|  1| 40|19410 Leafwood Ln...|  220000|Single family res...|  3| 1384|  34|      0|  52|
|  2|  0|13219 Liliana Gle...|  242500|Single family res...|  3| 1597|   8|      0|  26|
|  3|  0|13219 Liliana Gle...|  129900|               Ranch|  3| 1170|  11|     25|  28|
|  4| 17|4959 Culmore Dr, ...|  345000|Single family res...|  4| 1925|  50|     54|  56|
|  5|  0|13219 Liliana Gle...|  260000|Single family res...|  3| 1296|  75|     43|  60|
|  6| 62|803 Martin Rd A, ...|  449900|Single family res...|  3| 1861|  39|     47|  43|
|  7| 77|1806 Royal Fern C...| 1089500|Single family res...|  5| 5456|  24|      0|  44|
|  8| 85|5850 Petty S

In [192]:
#rename the first two colums to fit the sql table name scheme
df_properties_rename = df_properties.withColumnRenamed("id", "super_id")
df_properties_rename2 = df_properties_rename.withColumnRenamed("_c0", "id")
df_properties_rename2.show()

+---+--------+--------------------+--------+--------------------+---+-----+----+-------+----+
| id|super_id|             address|   price|                type|bed| sqft|walk|transit|bike|
+---+--------+--------------------+--------+--------------------+---+-----+----+-------+----+
|  0|      80|4031 Wilderness F...|  369900|Single family res...|  2| 2395|  53|      0|  47|
|  1|      40|19410 Leafwood Ln...|  220000|Single family res...|  3| 1384|  34|      0|  52|
|  2|       0|13219 Liliana Gle...|  242500|Single family res...|  3| 1597|   8|      0|  26|
|  3|       0|13219 Liliana Gle...|  129900|               Ranch|  3| 1170|  11|     25|  28|
|  4|      17|4959 Culmore Dr, ...|  345000|Single family res...|  4| 1925|  50|     54|  56|
|  5|       0|13219 Liliana Gle...|  260000|Single family res...|  3| 1296|  75|     43|  60|
|  6|      62|803 Martin Rd A, ...|  449900|Single family res...|  3| 1861|  39|     47|  43|
|  7|      77|1806 Royal Fern C...| 1089500|Single family re

In [193]:
#Check the data types of columns
df_properties_rename2.printSchema

<bound method DataFrame.printSchema of DataFrame[id: int, super_id: int, address: string, price: int, type: string, bed: string, sqft: string, walk: int, transit: int, bike: int]>

In [194]:
# Import data types
from traitlets.traitlets import Float
from locale import currency
from pyspark.sql.types import *

#cast sqft and bed as int datatypes
df_cast = df_properties_rename2.withColumn("sqft", df_properties_rename2["sqft"].cast(IntegerType()))
df_cast2 = df_cast.withColumn("bed", df_properties_rename2["bed"].cast(IntegerType()))




In [195]:
#Check datatypes again
#all correct
df_cast2.printSchema

<bound method DataFrame.printSchema of DataFrame[id: int, super_id: int, address: string, price: int, type: string, bed: int, sqft: int, walk: int, transit: int, bike: int]>

In [196]:
# show new dataframe
df_cast2.show()

+---+--------+--------------------+--------+--------------------+---+-----+----+-------+----+
| id|super_id|             address|   price|                type|bed| sqft|walk|transit|bike|
+---+--------+--------------------+--------+--------------------+---+-----+----+-------+----+
|  0|      80|4031 Wilderness F...|  369900|Single family res...|  2| 2395|  53|      0|  47|
|  1|      40|19410 Leafwood Ln...|  220000|Single family res...|  3| 1384|  34|      0|  52|
|  2|       0|13219 Liliana Gle...|  242500|Single family res...|  3| 1597|   8|      0|  26|
|  3|       0|13219 Liliana Gle...|  129900|               Ranch|  3| 1170|  11|     25|  28|
|  4|      17|4959 Culmore Dr, ...|  345000|Single family res...|  4| 1925|  50|     54|  56|
|  5|       0|13219 Liliana Gle...|  260000|Single family res...|  3| 1296|  75|     43|  60|
|  6|      62|803 Martin Rd A, ...|  449900|Single family res...|  3| 1861|  39|     47|  43|
|  7|      77|1806 Royal Fern C...| 1089500|Single family re

In [199]:
# Write edited df to properties table in RDS
df_cast2.write.jdbc(url=jdbc_url, table='properties', mode=mode, properties=config)

In [200]:
# Read properties from table
read_df_properties = spark.read.jdbc(url=jdbc_url, table='properties', properties=config)
read_df_properties.show()

+---+--------+--------------------+--------------+--------------------+---+-----+----+-------+----+
| id|super_id|             address|         price|                type|bed| sqft|walk|transit|bike|
+---+--------+--------------------+--------------+--------------------+---+-----+----+-------+----+
|  0|      80|4031 Wilderness F...|   $369,900.00|Single family res...|  2| 2395|  53|      0|  47|
|  1|      40|19410 Leafwood Ln...|   $220,000.00|Single family res...|  3| 1384|  34|      0|  52|
|  2|       0|13219 Liliana Gle...|   $242,500.00|Single family res...|  3| 1597|   8|      0|  26|
|  3|       0|13219 Liliana Gle...|   $129,900.00|               Ranch|  3| 1170|  11|     25|  28|
|  4|      17|4959 Culmore Dr, ...|   $345,000.00|Single family res...|  4| 1925|  50|     54|  56|
|  5|       0|13219 Liliana Gle...|   $260,000.00|Single family res...|  3| 1296|  75|     43|  60|
|  6|      62|803 Martin Rd A, ...|   $449,900.00|Single family res...|  3| 1861|  39|     47|  43|


In [ ]:
#Read superneighborhoods table
read_df.show()

+---+--------------------+----------------+-----------------------+
| id|                name|total_population|median_household_income|
+---+--------------------+----------------+-----------------------+
|  0|                 N/A|           24819|                  30438|
|  1|    Kashmere Gardens|           10055|                  23102|
|  2|Trinity / Houston...|           15789|                  27789|
|  3|           Settegast|            2981|                  28821|
|  4|Denver Harbor / P...|           17571|                  29273|
|  5|          South Park|           21208|                  29475|
|  6|            Westwood|           19530|                  29527|
|  7|  Greater Fifth Ward|           19687|                  30535|
|  8|Clinton Park / Tr...|            3140|                  31826|
|  9|        East Houston|           18580|                  31826|
| 10|       Magnolia Park|           16999|                  32039|
| 11|Gulfgate Rivervie...|           12723|     

In [208]:
df_joined = read_df.join(read_df_properties, read_df.id == read_df_properties.super_id, "inner").select(read_df_properties["super_id"],read_df["name"],
                                                                                                        read_df["total_population"],
                                                                                                        read_df["median_household_income"],
                                                                                                        read_df_properties["address"],
                                                                                                        read_df_properties["price"],
                                                                                                        read_df_properties["type"],
                                                                                                        read_df_properties["bed"],
                                                                                                        read_df_properties["sqft"],
                                                                                                        read_df_properties["walk"],
                                                                                                        read_df_properties["transit"],
                                                                                                        read_df_properties["bike"]
                                                                                                        )
df_joined.show()

+--------+--------------------+----------------+-----------------------+--------------------+-------------+--------------------+---+----+----+-------+----+
|super_id|                name|total_population|median_household_income|             address|        price|                type|bed|sqft|walk|transit|bike|
+--------+--------------------+----------------+-----------------------+--------------------+-------------+--------------------+---+----+----+-------+----+
|      31|  Greater Third Ward|           14295|                  40523|2410 Truxillo St,...|  $599,990.00|             Triplex|  3|3514|  77|     53|  83|
|      31|  Greater Third Ward|           14295|                  40523|2909 Cleburne St,...|  $500,000.00|              Duplex|  2|2046|  67|     52|  76|
|      31|  Greater Third Ward|           14295|                  40523|2518 Cleburne St,...|$1,749,900.00|              Duplex|  4|3096|  69|     50|  79|
|      31|  Greater Third Ward|           14295|                